In [1]:
Boyle.mk("matrex_samples")

All dependencies up to date


{:ok, ["matrex_samples"]}

In [2]:
Boyle.activate("matrex_samples")

All dependencies up to date


:ok

In [3]:
Boyle.install({:matrex, "~> 0.6"})

Resolving Hex dependencies...
Dependency resolution completed:
Unchanged:
  elixir_make 0.4.2
  matrex 0.6.7
All dependencies up to date
==> matrex
make: Nothing to be done for 'build'.


:ok

In [4]:
m = Matrex.magic(3)

#Matrex[3×3]
┌                         ┐
│     8.0     1.0     6.0 │
│     3.0     5.0     7.0 │
│     4.0     9.0     2.0 │
└                         ┘

In [5]:
m[2][3]

7.0

In [6]:
m[1..2]

#Matrex[2×3]
┌                         ┐
│     8.0     1.0     6.0 │
│     3.0     5.0     7.0 │
└                         ┘

In [7]:
m[:rows]

3

In [8]:
m[:size]

{3, 3}

In [9]:
m[:max]

9.0

In [10]:
m[2][:max]

7.0

In [11]:
m[:argmax]

8

In [12]:
m[2][:argmax]

3

In [13]:
import Matrex

defmodule LinearRegression do
  def lr_cost_fun(%Matrex{} = theta, {%Matrex{} = x, %Matrex{} = y, lambda} = _params, iteration \\ 0)
      when is_number(lambda) do
    m = y[:rows]

    h = Matrex.dot_and_apply(x, theta, :sigmoid)
    l = Matrex.ones(theta[:rows], theta[:cols]) |> Matrex.set(1, 1, 0)

    regularization =
      Matrex.dot_tn(l, Matrex.square(theta))
      |> Matrex.scalar()
      |> Kernel.*(lambda / (2 * m))

    # Compute the cost and add regularization parameter
    j =
      y
      |> Matrex.dot_tn(Matrex.apply(h, :log), -1)
      |> Matrex.subtract(
        Matrex.dot_tn(
          Matrex.subtract(1, y),
          Matrex.apply(Matrex.subtract(1, h), :log)
        )
      )
      |> Matrex.scalar()
      |> (fn
            :nan -> :nan
            x -> x / m + regularization
          end).()

    # Compute gradient
    grad =
      x
      |> Matrex.dot_tn(Matrex.subtract(h, y))
      |> Matrex.add(Matrex.multiply(theta, l), 1.0, lambda)
      |> Matrex.divide(m)

    {j, grad}
  end
  
  # The same cost function, implemented with  operators from `Matrex.Operators` module.
  # Works 2 times slower, than standard implementation. But it's a way more readable.
  # It is here for demonstrating possibilites of the library.
  def lr_cost_fun_ops(%Matrex{} = theta, {%Matrex{} = x, %Matrex{} = y, lambda} = _params)
      when is_number(lambda) do
    # Turn off original operators. Use this with caution!
    import Kernel, except: [-: 1, +: 2, -: 2, *: 2, /: 2, <|>: 2]
    import Matrex
    import Matrex.Operators
    
    # This line is needed only when used from iex, to remove ambiguity of t/1 function.
    import IEx.Helpers, except: [t: 1]

    m = y[:rows]

    h = sigmoid(x * theta)
    l = ones(size(theta)) |> set(1, 1, 0.0)

    j = (-t(y) * log(h) - t(1 - y) * log(1 - h) + lambda / 2 * t(l) * pow2(theta)) / m

    grad = (t(x) * (h - y) + (theta <|> l) * lambda) / m

    {scalar(j), grad}
  end
end

{:module, LinearRegression, <<70, 79, 82, 49, 0, 0, 15, 244, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 124, 0, 0, 0, 45, 23, 69, 108, 105, 120, 105, 114, 46, 76, 105, 110, 101, 97, 114, 82, 101, 103, 114, 101, 115, 115, 105, 111, ...>>, {:lr_cost_fun_ops, 2}}

In [14]:
x = Matrex.load("../matrex/test/data/X.mtx.gz")

#Matrex[5000×400]
┌                                                                             ┐
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     ⋮       ⋮       

In [15]:
x[1100..1115] |> list_of_rows() |> Enum.map(&(reshape(&1, 20, 20) |> transpose()))  |> reshape(4, 4) |> heatmap()

#Matrex[80×80]
┌                                                                                ┐
│         ▄▄▄                                                                    │
│       ▄▄▄▄▄▄▄▄                ▄ ▄▄▄▄             ▄▄▄▄▄▄        ▄▄▄▄▄▄▄▄▄       │
│       ▄▄▄▄▄▄▄▄▄             ▄▄▄▄▄▄▄▄▄           ▄▄▄▄▄▄▄▄▄       ▄     ▄▄▄▄     │
│        ▄▄▄▄ ▄ ▄▄            ▄▄▄▄ ▄▄▄▄         ▄▄ ▄▄▄▄▄▄                 ▄▄     │
│            ▄▄ ▄▄         ▄▄▄▄▄▄▄▄▄▄▄          ▄▄▄▄ ▄▄▄▄                 ▄▄     │
│    ▄▄▄▄▄▄▄▄▄ ▄▄       ▄▄▄▄▄▄▄▄▄▄▄▄▄▄        ▄▄▄▄▄▄▄▄▄                 ▄▄▄▄     │
│  ▄▄▄▄▄▄▄▄▄▄ ▄▄▄▄     ▄▄▄▄▄▄▄ ▄▄▄▄▄▄▄     ▄▄▄▄▄▄▄▄▄▄▄           ▄▄▄▄▄▄▄▄ ▄▄▄▄▄▄ │
│  ▄ ▄▄▄▄▄▄▄▄▄▄▄▄▄▄    ▄▄▄▄▄▄▄▄▄            ▄▄▄▄▄▄▄▄▄▄          ▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│
│  ▄▄▄▄▄▄▄ ▄             ▄▄▄▄▄                  ▄  ▄▄▄           ▄▄▄▄▄           │
│                                                                                │
│                            ▄▄                                         

#Matrex[80×80]
┌                                                                             ┐
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …   8.0e-5     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …  -0.0007     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  … -0.00269     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  … -0.00279     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  … -0.00278     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  … -0.00278     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  … -0.00113     0.0     0.0     0.0 │
│     ⋮       ⋮       ⋮  

In [16]:
y = Matrex.load("../matrex/test/data/Y.mtx")

#Matrex[5000×1]
┌         ┐
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│     ⋮   │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
└         ┘

In [17]:
theta = Matrex.zeros(x[:cols], 1)

#Matrex[400×1]
┌         ┐
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     ⋮   │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
└         ┘

In [18]:
lambda = 0.01
iterations = 100

100

In [19]:
solutions =
      1..10
      |> Task.async_stream(
        fn digit ->
          y3 = Matrex.apply(y, fn val -> if(val == digit, do: 1.0, else: 0.0) end)
          {sX, fX, _i} =
            Matrex.Algorithms.fmincg(&LinearRegression.lr_cost_fun/3, theta, {x, y3, lambda}, iterations)
          {digit, List.last(fX), sX}
        end,
        max_concurrency: 4
      )
      |> Enum.map(fn {:ok, {_d, _l, theta}} -> Matrex.to_list(theta) end)
      |> Matrex.new()

#Matrex[10×400]
┌                                                                             ┐
│     0.0     0.0  1.2e-4-0.00107-0.00117  … -0.07428 0.00924     0.0     0.0 │
│     0.0     0.0 -9.0e-5 0.00102 -6.2e-4  …  0.02237 0.00611 -0.0007     0.0 │
│     0.0     0.0 -5.0e-5-0.00108 0.02117  …  0.00117 -4.0e-5     0.0     0.0 │
│     0.0     0.0 -1.0e-5  6.0e-5  0.0013  … -0.00308  2.6e-4  1.0e-5     0.0 │
│     0.0     0.0     0.0  2.3e-4-0.00268  …  0.01584-0.00147     0.0     0.0 │
│     0.0     0.0     0.0  4.0e-5 -3.1e-4  …   0.0044 -4.7e-4  1.0e-5     0.0 │
│     0.0     0.0 -3.0e-5  0.0002   0.001  … -0.00276 -6.6e-4  8.0e-5     0.0 │
│     0.0     0.0 -6.0e-5  5.2e-4  5.9e-4  … -0.00499 -0.0035  3.7e-4     0.0 │
│     0.0     0.0     0.0  5.0e-5  1.5e-4  …   2.1e-4-0.00525  4.2e-4     0.0 │
│     0.0     0.0     0.0 -1.0e-5  1.6e-4  …  0.00163 -3.7e-4  1.0e-5     0.0 │
└                                                                             ┘

In [20]:
solutions =
      1..10  # Our ten digits, we wish to recognize
      |> Task.async_stream(
        fn digit ->
          # Prepare labels matrix with only current digit labeled with 1.0
          y3 = Matrex.apply(y, fn val -> if(val == digit, do: 1.0, else: 0.0) end)

          # Use fmincg() optimizer (ported to Elixir with Matrex functions) with previously defined cost function.
          {sX, fX, _i} =
            Matrex.Algorithms.fmincg(&LinearRegression.lr_cost_fun/3, theta, {x, y3, lambda}, iterations)
        
          # Return the digit itself and the best found solution, which is a column matrix 401x1
          {digit, List.last(fX), sX}
        end,
        max_concurrency: 4
      ) # Merge all 10 found solution column matrices into one 10x401 solutions matrix
      |> Enum.map(fn {:ok, {_d, _l, theta}} -> Matrex.to_list(theta) end)
      |> Matrex.new()

#Matrex[10×400]
┌                                                                             ┐
│     0.0     0.0  1.2e-4-0.00107-0.00117  … -0.07428 0.00924     0.0     0.0 │
│     0.0     0.0 -9.0e-5 0.00102 -6.2e-4  …  0.02237 0.00611 -0.0007     0.0 │
│     0.0     0.0 -5.0e-5-0.00108 0.02117  …  0.00117 -4.0e-5     0.0     0.0 │
│     0.0     0.0 -1.0e-5  6.0e-5  0.0013  … -0.00308  2.6e-4  1.0e-5     0.0 │
│     0.0     0.0     0.0  2.3e-4-0.00268  …  0.01584-0.00147     0.0     0.0 │
│     0.0     0.0     0.0  4.0e-5 -3.1e-4  …   0.0044 -4.7e-4  1.0e-5     0.0 │
│     0.0     0.0 -3.0e-5  0.0002   0.001  … -0.00276 -6.6e-4  8.0e-5     0.0 │
│     0.0     0.0 -6.0e-5  5.2e-4  5.9e-4  … -0.00499 -0.0035  3.7e-4     0.0 │
│     0.0     0.0     0.0  5.0e-5  1.5e-4  …   2.1e-4-0.00525  4.2e-4     0.0 │
│     0.0     0.0     0.0 -1.0e-5  1.6e-4  …  0.00163 -3.7e-4  1.0e-5     0.0 │
└                                                                             ┘

In [21]:
predictions =
  x
  |> Matrex.dot_nt(solutions)
  |> Matrex.apply(:sigmoid)

#Matrex[5000×10]
┌                                                                             ┐
│     0.0  1.8e-4  8.0e-5     0.0  1.4e-4  …      0.0  1.5e-4 0.00148 0.99992 │
│     0.0  1.0e-5  7.0e-5     0.0 0.00637  …      0.0     0.0  1.0e-5 0.99995 │
│     0.0  1.1e-4  3.0e-5     0.0  1.9e-4  …      0.0 0.00972 0.00116  0.9976 │
│     0.0  1.2e-4  1.0e-5     0.0     0.0  …      0.0  6.0e-5  1.7e-4 0.99986 │
│     0.0     0.0  1.0e-5     0.0 0.00167  …      0.0  1.0e-5     0.0 0.98872 │
│     0.0  5.0e-5     0.0     0.0  1.0e-5  …      0.0 0.00696  1.0e-5     1.0 │
│     0.0     0.0 0.01038     0.0     0.0  …      0.0  1.0e-5  1.0e-5  0.9493 │
│     0.0 0.37895 0.00361     0.0 0.11678  …      0.0  0.0175     0.0  0.9214 │
│     0.0  3.0e-5  1.1e-4     0.0  3.1e-4  …      0.0 0.00435  5.0e-5 0.99452 │
│     0.0     0.0 0.23053     0.0 0.00105  …      0.0  1.0e-5 0.00154 0.99776 │
│     0.0     0.0  0.0005     0.0 0.15892  …      0.0 0.00191  2.8e-4 0.98208 │
│     ⋮       ⋮       ⋮

In [22]:
accuracy =
  1..predictions[:rows]
  |> Enum.reduce(0, fn row, acc ->
    if y[row] == predictions[row][:argmax], do: acc + 1, else: acc
  end)
  |> Kernel./(predictions[:rows])
  |> Kernel.*(100)

95.5